In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [17]:
# import all excel datasets using pandas
# excel files for the first set of data
# tsangkaris
sum_table1 = pd.read_excel("Supplementary Table 8.xlsx")
CB_1 = pd.read_excel("Mouse_Brain_-_Cerebellum_7_weeks.xlsx")
CC_1 = pd.read_excel("Mouse_Brain_-_Cortex_7_weeks.xlsx")
HT_1 = pd.read_excel("mouse_brain_-_hipothalamus_7_weeks-01.xlsx")
HC_1 = pd.read_excel("mouse_brain_-_hipocampus_7_weeks-01.xlsx")
MD_1 = pd.read_excel("mouse_brain_-_medulla_7_weeks_old-01.xlsx")
MB_1 = pd.read_excel("Mouse_Brain_-_Mid_Brain_7_weeks.xlsx")
OB_1 = pd.read_excel("mouse_brain_-_olfactory_balb_7_weeks-01.xlsx")

# taraslia
sum_table2 = pd.read_excel("TARASLIA et al TABLE 1.xls", skiprows=1)
CB_2 = pd.read_excel("TARASLIA et al TABLE 6.xls", skiprows=1)
CC_2 = pd.read_excel("TARASLIA et al TABLE 8.xls", skiprows=1)
HT_2 = pd.read_excel("TARASLIA et al TABLE 7.xls", skiprows=1)
HC_2 = pd.read_excel("TARASLIA et al TABLE 3.xls", skiprows=1)
MD_2 = pd.read_excel("TARASLIA et al TABLE 5.xls", skiprows=1)
MB_2 = pd.read_excel("TARASLIA et al TABLE 4.xls", skiprows=1)
OB_2 = pd.read_excel("TARASLIA et al TABLE 2.xls", skiprows=1)


First of all try to change the column names so that they are all the same for both origins.

In [18]:
# replace in series 1 tables the accession with entry name
mapping_df = sum_table1[['Entry', 'Entry name']].copy()

dfs_series1 = [CB_1, CC_1, HT_1, HC_1, MD_1, MB_1, OB_1]

for df in dfs_series1:
    # Use the map function to replace values in the 'Accession' column
    df['Accession'] = df['Accession'].map(mapping_df.set_index('Entry')['Entry name'])

# rename the 'entry name' in the supplementary table 1
sum_table1.rename(columns={'Entry name':'Accession name'}, inplace=True)
sum_table1.drop('Entry', axis=1, inplace=True)

In [19]:
# create lists of the dataframes from the two different sources
dfs_series1 = [CB_1, CC_1, HT_1, HC_1, MD_1, MB_1, OB_1]
dfs_series2 = [CB_2, CC_2, HT_2, HC_2, MD_2, MB_2, OB_2]

# convert Da to kDa in 2nd series of documents and rename the columns
for df in dfs_series2:
    try:
        df['Protein MW (Da)'] = df['Protein MW (Da)'] / 1000
        df.rename(columns={'Protein MW (Da)': 'Molecular weight (kDa)',
                           'pI-value':'pI-Value'}, inplace=True)
    except KeyError:
        # Handle the case when 'Protein MW (Da)' column is not present
        pass

# renaming the column names of the first series
for df in dfs_series1:
    df.rename(columns={'Score': 'Mascot Score',
                       'Accession': 'Accession name',
                       'Coverage':'Coverage (%)',
                       '# Proteins': 'Proteins',
                       '# Unique Peptides': 'Unique Peptides',
                       '# Peptides':'Peptides',
                       '# PSMs':'PSMs',
                       '# AAs': 'Amino Acids',
                       'MW [kDa]': 'Molecular weight (kDa)',
                       'calc. pI': 'pI-Value'}, inplace=True)


In [20]:

sum_table1.head()

,Accession name,Protein names,Cerebral cortex,Olfactory Bulb,Hippocampus,Hypothalamus,Mid brain,Cerebellum,Medulla
0,A0A067XG52_MOUSE,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,A0A4W9_MOUSE,Neuronal growth regulator 1,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,SMKY_MOUSE,Sperm motility kinase Y (EC 2.7.11.1),0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,A0JLR7_MOUSE,Ahnak protein (Fragment),0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
sum_table2.head()

,Accession Name,Protein name,Protein MW,pI-Value,Brain part,Subcellular localisation,Function
0,1433B_MOUSE,14-3-3 protein beta/alpha,28183,4.62,"OB, HT, MD",Cytoplasm,"Transcription rergulation, cytoplasmic sequest..."
1,1433E_MOUSE,14-3-3 protein epsilon,29326,4.50,"OB, MB, HT, HC, CB, MD",Cytoplasm,"Cerebral cortex development, hippocampus devel..."
2,1433F_MOUSE,14-3-3 protein eta,28365,4.66,OB,Cytoplasm,"Glucocorticoid catabolism, glucocorticoid rece..."
3,1433G_MOUSE,14-3-3 protein gamma,28456,4.70,"OB, CC, MB, HT, HC, CB, MD",Cytoplasm,"Neuronal differentiation, regulation of synapt..."
4,1433T_MOUSE,14-3-3 protein theta,28046,4.54,"OB, MD",Cytoplasm,"Trasnscription, signal transduction"


In [22]:
CB_1.head()

,Accession name,Description,Mascot Score,Coverage (%),Proteins,Unique Peptides,Peptides,PSMs,Area,Amino Acids,Molecular weight (kDa),pI-Value
0,SPTN1_MOUSE,"Spectrin alpha chain, non-erythrocytic 1 OS=Mu...",656.608884,42.64,1,80,81,256,2.904653e+08,2472,284.422297,5.325684
1,KCRB_MOUSE,Creatine kinase B-type OS=Mus musculus GN=Ckb ...,299.488654,50.39,1,15,15,112,8.276152e+08,381,42.686280,5.668457
2,AT1A3_MOUSE,Sodium/potassium-transporting ATPase subunit a...,270.776774,28.83,3,13,21,106,1.501888e+09,1013,111.619932,5.414551
3,AT1A2_MOUSE,Sodium/potassium-transporting ATPase subunit a...,243.238156,28.82,3,15,23,90,6.516163e+08,1020,112.145477,5.554199
4,VDAC1_MOUSE,Voltage-dependent anion-selective channel prot...,221.960705,49.32,1,9,10,77,7.135767e+08,296,32.331378,8.426270


In [23]:
CB_2.head()

,Accession Name,Protein name,Mascot Score,Coverage (%),Molecular weight (kDa),pI-Value
0,4EBP2_MOUSE,Eukaryotic translation initiation factor 4E-bi...,55,80,13.004,6.10
1,CB070_MOUSE,UPF0573 protein C2orf70 homolog,57,43,23.113,10.06
2,CYC_MOUSE,"Cytochrome c, somatic",55,39,11.712,10.19
3,ENOB_MOUSE,Beta-enolase,58,33,47.337,6.89
4,PHLB2_MOUSE,Pleckstrin homology-like domain family B membe...,57,15,142.253,8.26
